In [44]:
#make corpus from nltk stuff(won't have to preprocess much)
#train glove on that corpus
#define functions you need
#what do you need bruhh
#make that nice co-occurrence matrix
#figure out how optimisation works
#figure out initiallisation
#figure out training shizz
#figure out how you exporting yo model and shizz


In [61]:
from scipy import sparse
import numpy as np
from keras.layers import Input, Embedding, Dot, Reshape, Add
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K
import nltk

In [62]:
import nltk
from nltk.corpus import brown

In [63]:
nltk.download('brown')
data = brown.sents(categories=brown.categories())
len(data)

[nltk_data] Downloading package brown to /home/shiven/nltk_data...
[nltk_data]   Package brown is already up-to-date!


57340

In [64]:
sentences=[]
stopwords_ = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
punctuation = ['!','"','#','$','%','&',"'",'(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[','\\',']','^','_','`','{','|','}','~','``',"''",'--']
for sentence in data:
    for word in stopwords_:
        token=" "+word+" "
        sentence=[item.replace(token," ") for item in sentence]
    sentences.append(sentence)


In [65]:
for i in range(len(sentences)):
    sentences[i]=[item.lower() for item in sentences[i]]
    sentences[i]=[item*(len(item)>2) for item in sentences[i]]
    for pun in punctuation:
        sentences[i]=[item.replace(pun,"") for item in sentences[i]]
        sentences[i]=[item for item in sentences[i] if item]

In [66]:
print(sentences[100])

['daniel', 'personally', 'led', 'the', 'fight', 'for', 'the', 'measure', 'which', 'had', 'watered', 'down', 'considerably', 'since', 'its', 'rejection', 'two', 'previous', 'legislatures', 'public', 'hearing', 'before', 'the', 'house', 'committee', 'revenue', 'and', 'taxation']


In [67]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [77]:
vocab_size=10000
vector_dim=10
maxlen=20
windowSize=3

In [78]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

In [79]:
sequences = tokenizer.texts_to_sequences(sentences)
sequences_padded = pad_sequences(sequences, padding='post', maxlen=maxlen)


In [80]:
print(sentences[4000])
print(sequences[4000])
print(sequences_padded[4000])
print(len(sentences[4000]))
print(len(sequences[4000]))
print(len(sequences_padded[4000]))

['ruths', 'day', 'and', 'until', 'this', 'year', 'the', 'schedule', 'was', '154', 'games']
[9177, 108, 3, 161, 9, 117, 2, 2968, 5, 1, 1826]
[9177  108    3  161    9  117    2 2968    5    1 1826    0    0    0
    0    0    0    0    0    0]
11
11
20


In [81]:
from collections import defaultdict
co_matrix = defaultdict(lambda: defaultdict(int))

In [82]:
for sentence in sequences_padded:
    sentence_size=len(sentence)
    for i in range(sentence_size):
        for distance in range(1,windowSize+1):
            if i+distance<sentence_size:
                if sentence[i]>sentence[i+distance]:
                    first=sentence[i+distance]
                    second=sentence[i]
                else:
                    second=sentence[i+distance]
                    first=sentence[i]
                co_matrix[first][second]+=1.0/distance

In [83]:
first,second,freqs=[],[],[]
for first_id in co_matrix.keys():
    for second_id in co_matrix[first_id].keys():
        freq=co_matrix[first_id][second_id]
        first.append(first_id)
        second.append(second_id)
        freqs.append(freq)
        first.append(second_id)
        second.append(first_id)
        freqs.append(freq)
first=np.array(first)
second=np.array(second)
freqs=np.array(freqs)

In [75]:
print(np.shape(first))
print(np.shape(second))
print(np.shape(freqs))

(1306226,)
(1306226,)
(1306226,)


In [88]:
#time for modelling yay!
#shamelessly copied loss haha
X_MAX = 100
a = 3.0 / 4.0
def customLoss(y_true,y_pred):
    return K.sum(K.pow(K.clip(y_true / X_MAX, 0.0, 1.0), a) * K.square(y_pred - K.log(y_true)), axis=-1)

In [92]:
def modelMake():

    input_target = Input((1,))
    input_context = Input((1,))

    central_embedding = Embedding(vocab_size, vector_dim, input_length=1)
    central_bias = Embedding(vocab_size, 1, input_length=1)

    context_embedding = Embedding(vocab_size, vector_dim, input_length=1)
    context_bias = Embedding(vocab_size, 1, input_length=1)

    vector_target = central_embedding(input_target)
    vector_context = context_embedding(input_context)

    bias_target = central_bias(input_target)
    bias_context = context_bias(input_context)

    dot_product = Dot(axes=-1)([vector_target, vector_context])
    dot_product = Reshape((1, ))(dot_product)
    bias_target = Reshape((1,))(bias_target)
    bias_context = Reshape((1,))(bias_context)

    prediction = Add()([dot_product, bias_target, bias_context])

    model = Model(inputs=[input_target, input_context], outputs=prediction)
    model.compile(loss=customLoss, optimizer=Adam())

    return model

In [93]:
epochs=1
batch_size=1024
model=modelMake()

In [94]:
model.fit([first, second], freqs, epochs=50, batch_size=batch_size)

Epoch 1/1
1306226/1306226 [==============================] - 9s 7us/step - loss: 0.1008
